In [156]:
import pandas as pd

# Get the attendance file from Google sheets into python and do some initial cleanup
doc_df = pd.read_csv("doc_details.csv")

doc_df

,course name,sheet id,db sheet id,name column name
0,NYU_Corporate_Finance,1hadxCX_bBp9b8pgwta9L0o6jggmGFHWOpQT9srwoSFg,1HKKsmJPmx1y_jNl8Q_RPMW9jVQd_enTJREaD1oxwVnk,Please add your alpha numeric NYU (NetID) user...
1,NYU_Financial_Accounting,1HO7C4iTMINSonZAOl9L_v44P0GhV-bjfv50cw5GitEE,1Jqm43d9rXdUr2vBKUuZWi6mrPgI-sOpzV59lCPGClxA,Add your exact alpha numeric(NetID) NYU user n...
2,CUNY_Corporate_Finance,1IP9rkHoW75Q4F77luvTTnOxDUg5-ocQonbSSW7igIPk,1m3i5FQx5x1svVnHH1yfFXbiYVtABXbcJgNZyUdM4c5o,Add your exact numeric numeric-digit CUNY ID#.
3,CUNY_Futures_Options,1w6YGovBDZoll0wbzni12kIwBVuQsFjddG_du_YD3s_w,1WJa0KiUCrZPs19EwDugjtWwQ8Nmf--1RnrCLdyJAT9w,Add your exact numeric CUNY ID#.Pease eneter t...
4,CUNY_Portfolio_Managment,1wY3z05hbiSxdoN2vfwq_O7B6VQlzVoueOAmGKl0DJQI,1tuIFyZMyvncb2tm5FAn8znRl7bRRyV-2gu3UKFLSMkA,Student Name --Last name first (as appears exa...
5,FORDHAM_Multinational_Finance,1rFdsi9pIMUBSR43oBF6fMDv-kZd-kZ0g9Wr32WruUC8,14miHZy0R00wn8NFG2ZW5hKBDzaHT8LuDfDKOCMAVMKM,Student Name --Last name first (as appears exa...
6,SJU_Microeconomics,1oxdvfb3hHVUNpnhzbaGj4nqbs9QNeXr-xPezIEEUibE,1MSMK1vVQUo-wN5jzDWg1Nfu7g8OawTUK8oY4bnmDFlY,Student Name --Last name first (as appears exa...
7,SJU_Corporate_Finance,1tRDXiupRw7aa7heM-shF1zcL6G4qYd4GRFTets_TdiM,1GUOJsVlKRbEcyFk8m8ucuqK1WhGtBl0D2v-WG634qQc,Student Name --Last name first (as appears exa...


In [157]:
file_index = 5
sheet_id = doc_df.loc[file_index, "sheet id"]
doc_name = "https://docs.google.com/spreadsheets/export?id={}&exportFormat=csv".format(sheet_id)
df = pd.read_csv(doc_name)
df.rename(columns={df.columns[2]:"name", "Timestamp":"timestamp", "Email address":"email"}, inplace=True)


indices_to_drop = df[df["timestamp"]==' '].index
df.drop(index=indices_to_drop, inplace=True)
df.tail()

,timestamp,email,name,Unnamed: 3,Status
146,09/02/2024 13:29:28,sergiopad63@gmail.com,X03587665,NaN,NaN
147,09/02/2024 13:29:44,badizaman718@gmail.com,X03638117,NaN,NaN
148,09/02/2024 13:30:07,devin.spitz0531@gmail.com,"Spitz, Devin X03661113",NaN,NaN
149,09/02/2024 13:30:18,dmallas1127@gmail.com,X03590951,NaN,NaN
150,09/02/2024 13:30:39,clerge.tyrese@gmail.com,Clerge Tyrese X03668406,NaN,NaN


In [158]:

df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed', dayfirst=True)
#df['timestamp'] = pd.to_datetime(df['timestamp'], format="%d/%m/%Y %H:%M:%S")
df["name"] = df["name"].str.upper()
df["name"] = df["name"].replace(","," ", regex=True)
df["name"] = df["name"].replace("--"," ", regex=True)
df["name"] = df["name"].replace("-"," ", regex=True)
df["name"] = df["name"].replace("  "," ", regex=True)
df.tail()

,timestamp,email,name,Unnamed: 3,Status
146,2024-02-09 13:29:28,sergiopad63@gmail.com,X03587665,NaN,NaN
147,2024-02-09 13:29:44,badizaman718@gmail.com,X03638117,NaN,NaN
148,2024-02-09 13:30:07,devin.spitz0531@gmail.com,SPITZ DEVIN X03661113,NaN,NaN
149,2024-02-09 13:30:18,dmallas1127@gmail.com,X03590951,NaN,NaN
150,2024-02-09 13:30:39,clerge.tyrese@gmail.com,CLERGE TYRESE X03668406,NaN,NaN


In [159]:
# Get the student database file from Google sheets
db_doc_id = doc_df.loc[file_index, "db sheet id"]
db_doc_name = "https://docs.google.com/spreadsheets/export?id={}&exportFormat=csv".format(db_doc_id)
db = pd.read_csv(db_doc_name, keep_default_na=False)
db.head()

,Name,Other names,Email IDs,Student ID
0,ABUSO STEPHEN,,,
1,AL BAYATI ABDULLAH,,,
2,ALEXIOS GEORGIADES,,,
3,AMARI CHARLES,,,
4,ARANBAYEV EMILY,,,


In [160]:
# Remove duplicates
db.drop_duplicates(inplace=True)
indexes_to_drop = []
for index, ref_data in db.iterrows():
    ref_name_set = set(ref_data["Name"].split())
    for cindex, comp_data  in db.iterrows():
        comp_name_set = set(comp_data["Name"].split())
        if ref_name_set.issubset(comp_name_set):
            if index != cindex:
                indexes_to_drop.append(cindex)

db.drop(db.index[indexes_to_drop], inplace=True)

In [161]:
# Populate the student database
## Get email IDs of everyone
for index, ref_data in db.iterrows():
    ref_name_set = set(ref_data["Name"].split())
    for rindex, raw_data  in df.iterrows():
        raw_name_set = set(raw_data["name"].split())
        if ref_name_set.issubset(raw_name_set): # If the raw data first name, last name order is different, this will still work
            if len(ref_data["Email IDs"]) == 0:
                db.loc[index, "Email IDs"] = raw_data["email"]
                continue
            if set([raw_data["email"]]).issubset(set([ref_data["Email IDs"]])):     # If we already have the email ID in the student database, we ignore it
                continue
            else:
                db.loc[index, "Email IDs"] = db.loc[index, "Email IDs"]+" "+ raw_data["email"]

In [162]:
# Create the attendance record
unique_dates = list(df["timestamp"].dt.date.unique())
unique_dates_str = list(df["timestamp"].dt.strftime(date_format="%d-%m-%Y").unique())
unique_dates_str = list(filter(lambda x: type(x) == str , unique_dates_str))    # Remove nan
ar_col_names = ["Name", "Email IDs", "Attendance (%)"] + unique_dates_str
ar = pd.DataFrame(columns=ar_col_names)

ar["Name"] = db["Name"]
ar["Email IDs"] = db["Email IDs"]
ar["Attendance (%)"] = 0

# Record attendence for each class
for col_index, timestamp in enumerate(unique_dates):
    email_list = list(df.loc[df["timestamp"].dt.date == timestamp, "email"])
    for index, ar_data in ar.iterrows():
        if len(set(ar_data["Email IDs"].split(" ")).intersection(set(email_list))) > 0:
            ar.loc[index, unique_dates_str[col_index]] = "Y"
            ar.loc[index, "Attendance (%)"] = ar.loc[index, "Attendance (%)"] + (100/len(unique_dates_str))



C:\Users\dinesh\AppData\Local\Temp\ipykernel_14688\3933747657.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '14.285714285714286' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ar.loc[index, "Attendance (%)"] = ar.loc[index, "Attendance (%)"] + (100/len(unique_dates_str))


In [163]:
final_op = ar.round({"Attendance (%)":1})
final_op.head()
final_op.to_csv(doc_df.loc[file_index, "course name"]+".csv", index=False)